https://github.com/traja-team/traja?tab=readme-ov-file

remove the negative value sfrom the plot!!!

In [13]:
import re
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.path import Path
import matplotlib.patches as patches

# ---------- read log ----------
x_vals, y_vals = [], []
csv_name = "log"

with open(csv_name + ".csv", "r", encoding="utf-8", errors="ignore") as f:
    lines = f.readlines()

pat = re.compile(r"X:\s*([-+]?\d*\.\d+|\d+)\s*cm,\s*Y:\s*([-+]?\d*\.\d+|\d+)\s*cm")
for line in lines[1:]:  # skip header
    m = pat.search(line)
    if m:
        x_vals.append(float(m.group(1)))
        y_vals.append(float(m.group(2)))

x = np.array(x_vals, dtype=float)
y = np.array(y_vals, dtype=float)

if len(x) == 0:
    raise RuntimeError("No X/Y data parsed. Check regex or file format.")

# ---------- hampel filter ----------
def hampel_filter(x, k=7, t=3.0):
    x = np.asarray(x, float)
    y = x.copy()
    n = len(x)

    for i in range(n):
        lo = max(i - k, 0)
        hi = min(i + k + 1, n)
        window = x[lo:hi]

        med = np.median(window)
        mad = np.median(np.abs(window - med))
        sigma = 1.4826 * mad

        if sigma > 0 and abs(x[i] - med) > t * sigma:
            y[i] = med
    return y

x = hampel_filter(x, k=7, t=3.0)
y = hampel_filter(y, k=7, t=3.0)

t = np.arange(len(x))

# ---------- helpers ----------
def stylize_axes(ax):
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)

def plot_trajectory_2d(ax, x, y, t, cmap="viridis"):
    n = len(x)
    verts = np.column_stack([x, y])
    codes = [Path.MOVETO] + [Path.LINETO] * (n - 1)
    path = Path(verts, codes)
    patch = patches.PathPatch(path, edgecolor="#999999", facecolor="none", lw=2.5, alpha=0.3)
    ax.add_patch(patch)

    sc = ax.scatter(x, y, c=t, s=6, cmap=cmap, alpha=0.85)

    pad_x = 0.05 * (x.max() - x.min() + 1e-9)
    pad_y = 0.05 * (y.max() - y.min() + 1e-9)
    ax.set_xlim(x.min() - pad_x, x.max() + pad_x)
    ax.set_ylim(y.min() - pad_y, y.max() + pad_y)

    ax.set_aspect("equal", adjustable="box")
    ax.set_xlabel("X [cm]")
    ax.set_ylabel("Y [cm]")
    stylize_axes(ax)
    return sc

# ---------- combined figure: 2D + 1D x(t), y(t) ----------
fig, (ax2d, axx, axy) = plt.subplots(
    3, 1, figsize=(8, 9),
    gridspec_kw={"height_ratios": [2.2, 1.0, 1.0]},
    constrained_layout=True
)

sc = plot_trajectory_2d(ax2d, x, y, t)
ax2d.set_title("2D Trajectory (Hampel only)")

cbar = fig.colorbar(sc, ax=ax2d, fraction=0.046, pad=0.04)
cbar.set_label("time / frame")

axx.plot(t, x)
axx.set_ylabel("X [cm]")
axx.set_xlabel("time / frame")
stylize_axes(axx)

axy.plot(t, y)
axy.set_ylabel("Y [cm]")
axy.set_xlabel("time / frame")
stylize_axes(axy)

out_png = f"{csv_name}_traj.png"
plt.savefig(out_png, dpi=300)
plt.close(fig)

print(f"Saved: {out_png}")


Saved: log_traj.png
